In [6]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import urllib.request
from gensim.models.word2vec import Word2Vec

In [3]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt', filename='ratings.txt')


('ratings.txt', <http.client.HTTPMessage at 0x7f992666bed0>)

In [4]:
train_data = pd.read_table('ratings.txt')
train_data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [12]:
train_data = train_data.dropna()

In [13]:
train_data['clean_doc'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]", '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
stw = pd.read_csv('/content/drive/MyDrive/stopwords-ko.txt', header = None)
stw_list = list(stw[0].values)

In [15]:
from konlpy.tag import Mecab

In [16]:
mecab = Mecab()
tokenized_data = []

for sentence in train_data['clean_doc']:
  temp_x = mecab.morphs(sentence)
  temp_x = [word for word in temp_x if not word in stw_list]
  tokenized_data.append(temp_x)

In [ ]:
tokenized_data

In [18]:
token_count = {}

for line in tokenized_data:
  for tok in line:
    if tok in token_count:
      token_count[tok] += 1
    else:
      token_count[tok] = 1

len(token_count)


69363

In [19]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size=100, window=5, min_count=5, sg=0)

In [20]:
model.wv.vectors.shape

(18409, 100)

In [21]:
model.wv.most_similar('최민식')

[('안성기', 0.906644880771637),
 ('한석규', 0.8772004246711731),
 ('전도연', 0.8680381774902344),
 ('박신양', 0.8607354164123535),
 ('정재영', 0.8590313196182251),
 ('송강호', 0.848669171333313),
 ('이범수', 0.843490481376648),
 ('최수종', 0.8401052951812744),
 ('설경구', 0.8400688767433167),
 ('드니로', 0.8369299173355103)]

In [22]:
model.wv.most_similar('무협')

[('느와르', 0.9245812296867371),
 ('sf', 0.8951829671859741),
 ('SF', 0.8854573965072632),
 ('호러', 0.871016263961792),
 ('헐리우드', 0.8628955483436584),
 ('할리우드', 0.8608379364013672),
 ('정통', 0.8476319313049316),
 ('블록버스터', 0.8458681702613831),
 ('멜로', 0.8452543616294861),
 ('첩보', 0.8413823843002319)]

In [23]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v')

In [24]:
loaded_model = KeyedVectors.load_word2vec_format('kor_w2v')

In [25]:
loaded_model.most_similar('tree')

KeyError: ignored

In [26]:
from gensim.models import FastText
fasttext_model = FastText(sentences = tokenized_data, size=100, window=5, min_count=5, sg=0)

In [27]:
fasttext_model.most_similar('dog')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('아씨', 0.8263850212097168),
 ('핏', 0.824290931224823),
 ('랄라', 0.8231834173202515),
 ('유희왕', 0.8141013383865356),
 ('제레미섬터', 0.809329628944397),
 ('아앙', 0.8077608346939087),
 ('수트', 0.8073720932006836),
 ('랄랄라', 0.8020764589309692),
 ('에용', 0.8015191555023193),
 ('교도', 0.8006157875061035)]

In [28]:
# pretrained Embedding

In [29]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

In [30]:
sample_text1 = 'bitty bought a bit of butter'
sample_text2 = 'but the bit of butter was a bit bitter'
sample_text3 = 'so she bought some better butter to make the bitter butter better'

corp = [sample_text1, sample_text2, sample_text3]
corp

['bitty bought a bit of butter',
 'but the bit of butter was a bit bitter',
 'so she bought some better butter to make the bitter butter better']

In [31]:
vocab_size = 50
encod_corp = []

for i, doc in enumerate(corp):
  encod_corp.append(one_hot(doc, 50))

encod_corp

[[6, 25, 29, 12, 11, 25],
 [5, 31, 12, 11, 25, 21, 29, 12, 32],
 [32, 19, 25, 45, 21, 25, 14, 15, 31, 32, 25, 21]]

In [32]:
model = Sequential()
model.add(Embedding(vocab_size, 128))

In [33]:
embeddings = model.layers[0].get_weights()[0]

In [34]:
embeddings.shape

(50, 128)

In [35]:
# 감성분류

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
sentences = [
             'nice great best amazing',
             'stop right lies',
             'pitiful nerd',
             'excellent work',
             'supreme bad',
             'highly respectable'
]

y_train = [1,0,0,1,1,0,1]

In [38]:
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

16


In [39]:
t.word_index

{'amazing': 4,
 'bad': 13,
 'best': 3,
 'excellent': 10,
 'great': 2,
 'highly': 14,
 'lies': 7,
 'nerd': 9,
 'nice': 1,
 'pitiful': 8,
 'respectable': 15,
 'right': 6,
 'stop': 5,
 'supreme': 12,
 'work': 11}

In [40]:
x_encoded = t.texts_to_sequences(sentences)

In [41]:
x_train = pad_sequences(x_encoded, maxlen=4)
y_train = np.array(y_train)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [44]:
model.fit(x_train, y_train, epochs=100, verbose=2)

ValueError: ignored

In [45]:
# 구글에서 사전에 훈련된 word2vec을 사용

In [46]:
# !wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [47]:
import tensorflow_hub as hub
import tensorflow as tf

In [48]:
# import gensim.downloader as api
# path = api.load('word2vec-google-news-300', return_path=True)

In [49]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable = True)

In [50]:
model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_1 (Dense)             (None, 16)                336       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [51]:
import gensim

In [53]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)

In [54]:
word2vec_model.vectors.shape

(3000000, 300)

In [55]:
# 16 * 300

embedding_matrix = np.zeros((vocab_size, 300))
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
# google w2v에서 해탕 토큰의 vector값을 추출

def get_vector(word):
  if word in word2vec_model:
    return word2vec_model[word]
  else:
    return None

In [57]:
for word, i in t.word_index.items():
  temp = get_vector(word)
  if temp is not None:
    embedding_matrix[i] = temp

In [58]:
embedding_matrix.shape

(16, 300)

In [59]:
from tensorflow.keras.layers import Input

In [ ]:
model = Sequential()
model.add(Input(shape=(4,), dtype='int32'))
e = Embedding(16, 300, weights=[embedding_matrix], trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [61]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [63]:
model.fit(x_train, y_train, epochs=100, verbose=2)

ValueError: ignored

In [64]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow as tf

In [66]:
(train_data, validation_data, test_data), info = tfds.load('imdb_reviews', split=['train[:80%]', 'train[80%:90%]',\
                                                                                  'train[90%:]'], with_info=True, as_supervised=True)

In [67]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [69]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [70]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable = False)
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [71]:
model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_4 (Dense)             (None, 16)                336       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 353
Non-trainable params: 400,020
_________________________________________________________________


In [72]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
hist = model.fit(train_data)